In [ ]:
!nohub

In [6]:
!nohup pip install wrapt --upgrade --ignore-installed
!nohup pip install tensorflow

nohup: ignoring input and appending output to ‘nohup.out’
nohup: ignoring input and appending output to ‘nohup.out’


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_treebank_pos_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping taggers/maxent_treebank_pos_tagger.zip.
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
# NLP
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# Classification
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

# Regression Models
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Recall, Precision
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Testing and optimization
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics.regression import mean_absolute_error, mean_squared_error, r2_score

# import module
from pipeline import *
from model import *

# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   
import pickle
import datetime as dt
import glob

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")


Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [5]:
# read csv from s3 bucket
s3 = boto3.resource('s3')
bucket = 'fakenewscorpus'
key = 'data/news_cleaned_2018_02_13.csv'
df = pd.read_csv('s3://{}/{}'.format(bucket,key), engine = 'python', nrows = 500000)

In [22]:
s3 = boto3.resource('s3')
bucket = 'fakenewscorpus'
key = 'data/5M_df.pkl'
# obj = s3.get_object(Bucket='bucket', Key='key')
df = pickle.loads(s3.Bucket(bucket).Object(key).get()['Body'].read())

In [6]:
def balance(df):
    n_pos = len(df[df['label']==1])
    n_neg = len(df[df['label']==0])
    return n_pos, n_neg

n_pos, n_neg = balance(df)

n_pos/len(df)

0.33273061724603664

In [7]:
sw = pd.read_csv('data/sw1k.csv')['term'].to_numpy()

sample = df.sample(100000,axis=0)

balance(sample)

(33258, 66742)

In [5]:
tokens = tokenize(sample['content'],sw)

sample['token']=tokens

X = tokens
y = sample['label']

X_train, X_test, y_train, y_test = train_test_split(X,y)

bow, tf, tfidf, cv, tv = vectorize(X_train,max_features=5000,ngram=1)

In [49]:
pd.DataFrame(tfidf).to_csv('data/sample_tfidf.csv')

In [9]:
tfidf = pd.read_csv('data/sample_tfidf.csv')

In [58]:
tfidf.shape

(7500, 5000)

In [60]:
MAX_SEQ_LENGHT = len(max(X_train, key=len))
MAX_SEQ_LENGHT 

5466

In [54]:
N_FEATURES = len(bow)
X_train_sequences = pad_sequences(X_train, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)

ValueError: invalid literal for int() with base 10: 'fsu'

In [21]:
t = Tokenizer(num_words = 500)
s = df['content']
t.fit_on_texts(s)
sq = t_temp.texts_to_sequences(r)
# print('sequences : ', sq,'\n')
# print('word_index : ',t_temp.word_index )

sequences :  [[22], [3], [2], [1], [], [3], [5], [12], [], [7], [14], [7], [3], [5], [5], [3], [], [2], [4], [4], [22], [], [3], [], [15], [4], [15], [], [3], [2], [], [2], [3], [17], [11], [4], [8], [], [3], [5], [12], [], [2], [4], [13], [], [], [16], [3], [8], [8], [3], [20], [3], [17], [], [20], [3], [7], [], [8], [1], [15], [4], [8], [2], [6], [5], [16], [], [2], [9], [1], [], [5], [1], [20], [7], [], [], [3], [5], [5], [4], [14], [5], [10], [6], [5], [16], [], [2], [9], [3], [2], [], [7], [20], [6], [18], [2], [], [9], [3], [7], [], [19], [1], [1], [5], [], [14], [5], [18], [4], [11], [11], [4], [20], [1], [12], [], [19], [17], [], [9], [1], [8], [], [1], [28], [], [19], [4], [17], [18], [8], [6], [1], [5], [12], [], [10], [3], [11], [21], [6], [5], [], [9], [3], [8], [8], [6], [7], [], [4], [5], [], [2], [20], [6], [2], [2], [1], [8], [], [], [], [8], [1], [15], [4], [8], [2], [7], [], [9], [3], [21], [1], [], [7], [14], [8], [18], [3], [10], [1], [12], [], [4], [21], [1], [8], 

In [5]:
df = pickle.load('data/5M_df.pkl')

TypeError: file must have 'read' and 'readline' attributes

In [7]:
df = clean(df)

In [9]:
X = df['content']
y = df['label']
t = Tokenizer(num_words=5000)
t.fit_on_texts(df['content'])
X = t.texts_to_sequences(df['content'])

In [10]:
X_matrix = pad_sequences(X, maxlen=5000)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_matrix,y)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [12]:
X_matrix.shape

(239504, 10000)

In [13]:
lstm_model = Sequential()

lstm_model.add(LSTM(units=64, return_sequences=True, input_shape=(1,5000)))
lstm_model.add(LSTM(64),)
lstm_model.add(Dense(units=1, activation='sigmoid'))
 
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC','BinaryAccuracy','Recall','Precision'])
print(lstm_model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 64)             2576640   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 2,609,729
Trainable params: 2,609,729
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
lstm_history = lstm_model.fit(X_train, y_train, 
          epochs=50, batch_size=128, verbose=1,
          validation_split=0.20)

In [ ]:
lstm_model.save('../saved_model/sample_lstm_model')

In [21]:
model = load_model('saved_model/sample_model',compile = False)
tfidf = pd.read_csv('data/sample_tfidf.csv')
X_train = tfidf.set_ind
y_train = sample['label']

In [25]:
lstm_model.evaluate(X_test, y_test, use_multiprocessing=True)

8309/8309 [==============================] - 1s 79us/sample - loss: 0.6295 - auc: 0.6636 - binary_accuracy: 0.6077 - recall: 0.7783 - precision: 0.6135


[0.6294791400324984, 0.66362023, 0.60765433, 0.7782875, 0.6134642]

In [29]:
Y_pred = lstm_model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [15]:
sw = pd.read_csv('../data/sw1k.csv')['term'].to_numpy()

df['token']=tokenize(df['content'],sw)


X = df['token']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y)

bow, tf, tfidf, cv, tv = vectorize(X_train,max_features=10000,ngram=1)

In [16]:
X_test = [' '.join(row) for row in X_test]

In [17]:
tfidf_test = tv.transform(X_test)

In [18]:
model2 = Sequential()
 
model2.add(Dense(units=500, activation='relu', input_dim=10000))
model2.add(Dense(units=1, activation='sigmoid'))
 
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC','BinaryAccuracy','Recall','Precision'])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               5000500   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 501       
Total params: 5,001,001
Trainable params: 5,001,001
Non-trainable params: 0
_________________________________________________________________


In [41]:
model2.fit(tfidf, y_train, 
          epochs=3, batch_size=128, verbose=1,
          validation_split=0.20)

Train on 21925 samples, validate on 3000 samples
Epoch 1/3
21925/21925 [==============================] - 1s 61us/sample - loss: 0.0015 - auc_2: 1.0000 - binary_accuracy: 0.9999 - recall_2: 0.9999 - precision_2: 0.9999 - val_loss: 0.5204 - val_auc_2: 0.9584 - val_binary_accuracy: 0.9293 - val_recall_2: 0.9297 - val_precision_2: 0.9401
Epoch 2/3
21925/21925 [==============================] - 1s 61us/sample - loss: 0.0049 - auc_2: 0.9999 - binary_accuracy: 0.9984 - recall_2: 0.9987 - precision_2: 0.9985 - val_loss: 0.6347 - val_auc_2: 0.9529 - val_binary_accuracy: 0.9290 - val_recall_2: 0.9224 - val_precision_2: 0.9461
Epoch 3/3
21925/21925 [==============================] - 1s 62us/sample - loss: 0.0851 - auc_2: 0.9948 - binary_accuracy: 0.9724 - recall_2: 0.9732 - precision_2: 0.9765 - val_loss: 0.2710 - val_auc_2: 0.9734 - val_binary_accuracy: 0.9280 - val_recall_2: 0.9462 - val_precision_2: 0.9237


In [ ]:
model2.save('../saved_model/sample_mlp_model2')

In [45]:
model2.evaluate(tfidf_test, y_test)

8309/8309 [==============================] - 0s 59us/sample - loss: 2.7775 - auc_2: 0.5000 - binary_accuracy: 0.4514 - recall_2: 0.0000e+00 - precision_2: 0.0000e+00


[2.7775004928128384, 0.5, 0.4514382, 0.0, 0.0]

In [ ]:
history = model2.fit(tfidf, y_train, 
          epochs=50, batch_size=128, verbose=1,
          validation_split=0.20)

143702/143702 [==============================] - 16s 114us/sample - loss: 0.0265 - auc_2: 0.9993 - binary_accuracy: 0.9936 - recall_2: 0.9824 - precision_2: 0.9967 - val_loss: 0.1983 - val_auc_2: 0.9704 - val_binary_accuracy: 0.9439 - val_recall_2: 0.8717 - val_precision_2: 0.9433
Epoch 6/50
143702/143702 [==============================] - 16s 114us/sample - loss: 0.0116 - auc_2: 0.9997 - binary_accuracy: 0.9979 - recall_2: 0.9942 - precision_2: 0.9989 - val_loss: 0.2201 - val_auc_2: 0.9676 - val_binary_accuracy: 0.9449 - val_recall_2: 0.8743 - val_precision_2: 0.9441
Epoch 7/50
143702/143702 [==============================] - 16s 114us/sample - loss: 0.0067 - auc_2: 0.9997 - binary_accuracy: 0.9990 - recall_2: 0.9977 - precision_2: 0.9990 - val_loss: 0.2435 - val_auc_2: 0.9653 - val_binary_accuracy: 0.9447 - val_recall_2: 0.8749 - val_precision_2: 0.9430
Epoch 8/50
143702/143702 [==============================] - 16s 115us/sample - loss: 0.0052 - auc_2: 0.9997 - binary_accuracy: 0.999

In [ ]:
def plot_history(history, metric):
# Plot training & validation accuracy values
    plt.plot(history.history[metric])
    plt.plot(history.history['val_{}'.format(metric)])
    plt.title('Model {}'.format(metric), size = 32)
    plt.ylabel('{}'.format(metric))
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [5]:
mlp_model = load_model('../saved_model/sample_mlp_model2')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
